<a href="https://colab.research.google.com/github/ssm951/chinese-genealogy/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OCR

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/Colab\ Notebooks/chinese-genealogy/

 bert-ancient-chinese-base.pt
'BERT Models.ipynb'
'Genealogy Book Deng Family Anrenli Village - Family Edition _ 安仁里鄧氏族譜 - 家庭版本 _ 安仁里邓氏族谱 - 家庭版本.pdf'
'Genealogy Book Deng Family Anrenli Village - Village Edition _ 安仁里鄧氏族譜 - 村版本 _ 安仁里邓氏族谱 - 村版本.pdf'
 Hong_Kong_Mo_genealogy_1999.pdf
 Meng_FamilySearch
 model.pt
 OCR.ipynb
 README.md
 siku-bert-base.pt


In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 66.3 MB/s eta 0:00:00


In [3]:
!pip install --upgrade google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.5/467.5 kB 17.2 MB/s eta 0:00:00


In [1]:
from google.colab import files
from google.cloud import vision
import fitz
from PIL import Image
import cv2
import io
import os
# only need to run once
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


Saving arcane-atom-206408-57eab0d8fd7e.json to arcane-atom-206408-57eab0d8fd7e.json
User uploaded file "arcane-atom-206408-57eab0d8fd7e.json" with length 2362 bytes


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = fn

In [39]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/chinese-genealogy/data/'
#sourcename = 'Hong_Kong_Mo_genealogy_1999'
#sourcename = 'Genealogy Book Deng Family Anrenli Village - Family Edition _ 安仁里鄧氏族譜 - 家庭版本 _ 安仁里邓氏族谱 - 家庭版本'
sourcename = 'Genealogy Book Deng Family Anrenli Village - Village Edition _ 安仁里鄧氏族譜 - 村版本 _ 安仁里邓氏族谱 - 村版本'
pdf_path = folder_path + sourcename + '/' + sourcename + '.pdf'

def convert_pdf_to_images(pdf_path):
    pdf_document = fitz.open(pdf_path)
    images = []
    for page in pdf_document:
      pix = page.get_pixmap()
      image = Image.open(io.BytesIO(pix.tobytes("png")))
      #image = image.rotate(-90, expand=True)
      images.append(image)
    return images

images = convert_pdf_to_images(pdf_path)

In [4]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/chinese-genealogy/data/'
sourcename = '吳氏族譜溯源記'
images_path = folder_path + sourcename + '/raw/'
images = []
for fn in os.listdir(images_path):
  image = Image.open(images_path + fn)
  images.append(image)

In [40]:
def detect_text_google_vision(image):
    client = vision.ImageAnnotatorClient()
    image_byte_array = io.BytesIO()
    image.save(image_byte_array, format='PNG')
    image_content = image_byte_array.getvalue()
    image = vision.Image(content=image_content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    if texts:
        return texts[0].description.replace(' ', '')
    return ''

texts_google = []
for i, image in enumerate(images):
    text = detect_text_google_vision(image)
    texts_google.append(text)

In [41]:
average = sum([len(text.split('\n')) for text in texts_google]) / len(texts_google)
processed_text = ""
for text in texts_google:
  lines = text.split('\n')
  for line in lines:
    line = line.strip()
    if len(line) == 0:
      continue
    elif all(char in '，、︒。°！？：；,.-1234567890 \t\n' for char in line):
      continue
    elif len(line) > 0.8 * average:
      processed_text += line
    else:
      processed_text += line + "\n"

In [42]:
processed_text = processed_text.replace('。', '。\n')
processed_text = processed_text.replace('︒', '︒\n')
processed_text = processed_text.replace('！', '！\n')
processed_text = processed_text.replace('？', '？\n')
processed_text = processed_text.replace('……', '……\n')
processed_text = processed_text.replace('；', '；\n')

In [43]:
print(processed_text)

安仁里鄧氏族譜
安仁里鄧氏族譜编纂領導小組
二〇〇八年四月
仁金量
鄧禹公像
2005年農歷二月
十五日安仁里文化
樓等六項工程剪彩
典禮盛况。

安仁里
安仁里文化樓
2008年農歷二月
初九日重修箕榮鄧
公祠竣工慶典盛
况。

捐助《安仁里鄧氏族譜》編纂、
出版經費的宗親的玉照、芳名
鄧燦良先生
美金200元
鄧真洽先生
美金200元
鄧祇雲先生
美金150元
鄧錦波先生
人民幣1000元
鄧柱培先生
美金100元
鄧金浣先生
美金100元
鄧連忠先生
美金100元
鄧德源先生
美金100元
鄧順富先生鄧柏昌先生
美金100元美金100元
鄧錦培先生美金50元
陳治香女士人民幣300元
鄧衛斌先生人民幣300元
鄧堅先生鄧銳富先生鄧權賢先生
美金100元美金100元美金100元
鄧伯球先生人民幣500元
鄧錦河先生人民幣300元
鄧焕明先生人民幣300元
鄧海波先生人民幣200元
望天管(土名)新建的樓房
村旁三棵茂盛的榕樹
《安仁里鄧氏族譜》编纂領導小組成員合
照,前排左起有源、尚志、毓培,後排左起富
榮、真和、金科、伯棠。

安仁里鄧氏族譜
安仁里鄧氏族譜編纂領導小組
二○○八年四月
安仁里鄧氏族譜編纂領導小組
組長:鄧尚志
副組長:鄧毓培鄧有源
成員:鄧金科鄧伯棠鄧真和鄧富榮
顧問:鄧錦波鄧燦良
鄧真洽鄧錦庭
鄧順波鄧連和
鄧柏玲
安仁里鄧氏族譜編寫組
主編:鄧毓培
副主編:鄧有源
編輯:鄧金科
安仁里鄧氏族譜
《安仁里鄧氏族譜》编纂領導小組
(廣東省台山市冲蔞鎮新屋安仁里)
廣東省台山市彩寧紙品印刷有限公司
(廣東省台山市中山路60號)
787×1092毫米16開本5印張86.6千字
印數1-500册
2008年5月第1版2008年5月第1次印刷
廣東省非營利性出版物準印證(内部)台字0856號
古序一
王羲之
周官小史,定世系辯昭穆之制。
上而君公,下及士大夫之家,不相越。
至我國朝摯虞以來,始有譜牒之作,考魏書所載譜系編,有四海大姓,姓、縣姓等紀,大都以門第相尚。
故魏書九品官人之法,多出于世族大家。
而世族之所通顯者,莫如南陽鄧氏為最著。

鄧氏自商王武丁封季父曼于河南南陽之國,遂以南陽為郡,以國為姓,分封食采,世襲侯爵,歷晁、徽、慶、桓、昆、忠、暉和、日浩、煦和、杞、熹、怡､沛、璧、淳、衡、略,至宣公字吾離,經一十九代。
周恒王時,離侯懼楚,與蔡

In [44]:
#os.mkdir(folder_path + sourcename)
with open(folder_path + sourcename + '/ocr_processed.txt', 'w') as f:
  f.write(processed_text)